In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import os
import platform

In [2]:
if platform.system()=='Windows': 
    #Directorios Windows
    os.chdir(
        r"G:\.shortcut-targets-by-id\120bUa35y3FJzvumWahpOx1qS9I6TIrhb\GIGANAV")

if platform.system()=='Darwin':
    os.chdir(
        '/Users/juliandiazparra/Library/CloudStorage/GoogleDrive-julian.adiaz@hotmail.com/Mi unidad/Negocios/Familiar/GIGANAV')

folder=os.getcwd()
data=os.path.join(folder,'Datos')
provisiones=os.path.join(data,'Provisiones')
facturas=os.path.join(data,'Facturas')
tickets=os.path.join(data,'Tickets')
usuarios=os.path.join(data,'Usuarios')
egresos=os.path.join(data,'Egresos')
transacciones=os.path.join(data,'Transacciones')
analisis=os.path.join(folder,'Operacion/Analisis')
resultados=os.path.join(analisis,'Resultados')
bases=os.path.join(analisis,'Bases_analisis')
outputs=os.path.join(analisis,'Outputs')
plots=os.path.join(outputs,'Gráficos')
usuarios_clean=os.path.join(bases,'Usuarios')
transac_clean=os.path.join(bases,'Transacciones')
tickets_clean=os.path.join(bases,'Tickets')
facturas_clean=os.path.join(bases,'Facturas')
egresos_clean=os.path.join(bases,'Egresos')
working=os.path.join(bases,'Bases_trabajo')
codigo=os.path.join(folder,'Operacion/Analisis/Código')
conciliaciones=os.path.join(folder,'Operacion/Operativo/Administrativo/Conciliaciones')

## Importación datos

## Tickets

In [3]:
#Tickets
years=[2022, 2023, 2024]
tickets_dict={}
tickets_append=pd.DataFrame()

for i in years:
    file_path=os.path.join(tickets_clean, 'Tickets_'+str(i))
    tickets_dict[i]=pd.read_csv(file_path+'.csv')
    tickets_append=pd.concat([
        tickets_dict[i],tickets_append],
                             ignore_index=True)
    
tickets_append.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15540 entries, 0 to 15539
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_ticket      15540 non-null  int64 
 1   departamento   15540 non-null  object
 2   nombre         15540 non-null  object
 3   asunto         15540 non-null  object
 4   tecnico        15540 non-null  object
 5   zona           15500 non-null  object
 6   abierto_por    15540 non-null  object
 7   instal         15540 non-null  int64 
 8   soporte        15540 non-null  int64 
 9   traslado       15540 non-null  int64 
 10  retiro         15540 non-null  int64 
 11  mantenimiento  15540 non-null  int64 
 12  facturacion    15540 non-null  int64 
 13  cambio_plan    15540 non-null  int64 
 14  suspendido     15540 non-null  int64 
 15  tv             15540 non-null  int64 
 16  llamada        15540 non-null  int64 
 17  categoria      14576 non-null  object
 18  fecha_abierto  15540 non-n

In [4]:
for i in ['fecha_abierto', 'fecha_cerrado']:
    tickets_append.loc[
        :,i]=pd.to_datetime(tickets_append[i], 
                            format='%Y-%m-%d %H:%M:%S')

#Creo mes y semana de cerrado
tickets_append.loc[
    :,'mes_crd']=pd.to_datetime(tickets_append[
    'fecha_cerrado']).dt.strftime('%Y-%m')

tickets_append.loc[
    :,'first_day']=tickets_append[
    'fecha_cerrado'] - pd.to_datetime(tickets_append[
    'fecha_cerrado']).dt.weekday * np.timedelta64(
    1, 'D')

tickets_append.loc[
    :,'first_day']=pd.to_datetime(tickets_append[
    'first_day']).dt.strftime(
    '%Y-%m-%d')

#Tiempo de resolucion
tickets_append.loc[
    :,'tiempo_resolucion']=tickets_append[
    'fecha_cerrado']-tickets_append[
    'fecha_abierto']

#Creo departamentos
tickets_append.loc[
    :, 'cat_tecnico']=tickets_append[
    'categoria'].str.contains(
    'Instalacion|Retiro|Soporte|Traslado|Mantenimiento', 
    case=False)*1

tickets_append.loc[
    (tickets_append['cat_tecnico']==1), 
    'departamento']='Tecnico'

tickets_append.loc[
    (tickets_append['categoria'].isna()), 
    'departamento']='Otro'

tickets_append.loc[
    :, 'comercial']=tickets_append[
    'categoria'].str.contains(
    'Facturacion|Cambio de plan|Suspendido', 
    case=False)*1
tickets_append.loc[
    tickets_append['comercial']==1, 
    'departamento']='Comercial'

/var/folders/_1/wccngqsd417dbsr4pcv82tj80000gn/T/ipykernel_9964/419886152.py:12: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  :,'first_day']=tickets_append[


## Usuarios

In [5]:
#Importo la base de usuarios más reciente que tenga
import glob
import os
list_of_files = glob.glob(os.path.join(usuarios_clean,'*.csv'))
latest_file = max(list_of_files, key=os.path.getctime)
base_usuarios=pd.read_csv(latest_file)

In [6]:
#Variables identificadoras usuarios
base_usuarios.loc[
    :,'tecnologia'
]=np.select([(
    base_usuarios['radio']==1)],['radio'], 
            default='fibra')

base_usuarios.loc[
    :,'tipo_servicio'
]=np.select([(
    base_usuarios['Tipo estrato']!=4)],['residencial'], 
            default='corporativo')

base_usuarios.loc[
    :,'tipo_usuario'
]=np.select([(
    base_usuarios['TV']==1)],['paquete'], 
            default='internet')

In [7]:
base_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3893 entries, 0 to 3892
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                3893 non-null   int64  
 1   nombre            3893 non-null   object 
 2   estado            3893 non-null   object 
 3   radio             3893 non-null   int64  
 4   Tipo estrato      3893 non-null   int64  
 5   megas             3893 non-null   int64  
 6   TV                3893 non-null   int64  
 7   Telefono          2919 non-null   object 
 8   Movil             3824 non-null   object 
 9   Correo            2958 non-null   object 
 10  Zona              3866 non-null   object 
 11  tarifa            3893 non-null   float64
 12  Deuda actual      1361 non-null   object 
 13  ID de registro    3847 non-null   float64
 14  meses_suspendido  320 non-null    float64
 15  deuda             320 non-null    float64
 16  tecnologia        3893 non-null   object 


## Material

In [8]:
spreadsheet_id = "1OoFik80Zcs6ih83j4_RrSdZAKaZNp9XyV-iMWiI1Vo4"
sheet_id = "26881671"
url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?gid={sheet_id}&format=csv"
uso_material = pd.read_csv(url)
uso_material.head()

,Marca temporal,Número de ticket,Técnicos involucrados,1. conector mecánico,2. Fibra óptica (metro),3.Conector coaxial,4. Tensores,5. cable coaxial,8. Deco TDT,9. Spliter 1*2,10. Spliter 1*3,11. Spliter 1*4,12. Mininodo,ADJUNTE POR FAVOR UNA FOTO DEL PERMISO DE TRABAJO EN ALTURAS (PTA),6.Cable UTP,7. RJs,Unnamed: 16,Unnamed: 17,deco,10. Otros
0,26/1/2024 11:01:19,NaN,NaN,2,78,6,5,16,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,27/1/2024 9:19:00,NaN,NaN,2,45,2,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27/1/2024 14:00:20,NaN,NaN,2,75,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27/1/2024 14:10:18,NaN,NaN,2,76,6,4,26,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27/1/2024 14:10:58,NaN,NaN,2,85,6,5,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#Limpio datos
#ID tickets
uso_material=uso_material.loc[
    uso_material['Número de ticket'].notna()]

uso_material.loc[:,'id_ticket'] =uso_material[
    'Número de ticket'].str.replace(
    '#','').str.lstrip('0').astype(int)

In [10]:
#Limpio datos
#ID tickets
uso_material=uso_material.loc[
    uso_material['Número de ticket'].notna()]

uso_material.loc[:,'id_ticket'] =uso_material[
    'Número de ticket'].str.replace(
    '#','').str.lstrip('0').astype(int)

#Separo tecnicos responsables
uso_material[[
    'tecnico_1', 'tecnico_2',
    'tecnico_3']]=uso_material[
    'Técnicos involucrados'].str.split(
    ',', expand=True).fillna('')

#Creo variable de mes de la fecha
uso_material.loc[:,'timestamp']=pd.to_datetime(uso_material['Marca temporal'], 
                                         format='%d/%m/%Y %H:%M:%S')
uso_material.loc[:,'month_pago']=uso_material['timestamp'].dt.strftime('%Y-%m')

base_material=uso_material.iloc[:,np.r_[20,21:23,3:8,14:16,8:13,19:20]]
base_material=base_material.fillna(0)
base_material.head()

,id_ticket,tecnico_1,tecnico_2,1. conector mecánico,2. Fibra óptica (metro),3.Conector coaxial,4. Tensores,5. cable coaxial,6.Cable UTP,7. RJs,8. Deco TDT,9. Spliter 1*2,10. Spliter 1*3,11. Spliter 1*4,12. Mininodo,10. Otros
7,189972,Sair Bolaño,Alfonso Arango,3,50,2,2,10,0,0,0,0,0.0,0.0,0,0.0
8,19313,Leonardo Quezada,,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0
9,19304,Sair Bolaño,Alfonso Arango,2,53,0,4,0,0,0,0,0,0.0,0.0,0,0.0
10,19312,Leonardo Quezada,,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0
11,19356,Leonardo Quezada,,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0


In [11]:
base_material.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1949 entries, 7 to 1955
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_ticket                1949 non-null   int64  
 1   tecnico_1                1949 non-null   object 
 2   tecnico_2                1949 non-null   object 
 3   1. conector mecánico     1949 non-null   object 
 4   2. Fibra óptica (metro)  1949 non-null   object 
 5   3.Conector coaxial       1949 non-null   object 
 6   4. Tensores              1949 non-null   object 
 7   5. cable coaxial         1949 non-null   object 
 8   6.Cable UTP              1949 non-null   object 
 9   7. RJs                   1949 non-null   object 
 10  8. Deco TDT              1949 non-null   object 
 11  9. Spliter 1*2           1949 non-null   object 
 12  10. Spliter 1*3          1949 non-null   float64
 13  11. Spliter 1*4          1949 non-null   float64
 14  12. Mininodo             1949

In [12]:
materiales_names=['conector_mecanico', 'fibra_drop', 'conector_coaxial', 
                  'tensores', 'cable_coaxial', 'cable_utp','rjs',
                  'deco_tdt', 'splitter_2', 'splitter_3', 'splitter_4', 
                  'mininodos', 'otros']

for x,y in enumerate(list(base_material.iloc[:,3:16].columns)):
    #Lleno vacios y elimino todos los caracteres no numericos
    base_material.loc[:,y]=base_material[y].astype(str).str.extract(r'(\d+)', expand=False).fillna(0).astype(int)
    base_material.rename(columns={y:materiales_names[x]}, inplace=True)

base_material.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1949 entries, 7 to 1955
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_ticket          1949 non-null   int64  
 1   tecnico_1          1949 non-null   object 
 2   tecnico_2          1949 non-null   object 
 3   conector_mecanico  1949 non-null   object 
 4   fibra_drop         1949 non-null   object 
 5   conector_coaxial   1949 non-null   object 
 6   tensores           1949 non-null   object 
 7   cable_coaxial      1949 non-null   object 
 8   cable_utp          1949 non-null   object 
 9   rjs                1949 non-null   object 
 10  deco_tdt           1949 non-null   object 
 11  splitter_2         1949 non-null   object 
 12  splitter_3         1949 non-null   float64
 13  splitter_4         1949 non-null   float64
 14  mininodos          1949 non-null   object 
 15  otros              1949 non-null   float64
dtypes: float64(3), int64(1), obje

## Encuesta satisfacción

In [13]:
spreadsheet_id = "1f5YsOXwh52gKB_qyll0D5GR8AbYrGzhJSgIAB_PrC4s"
sheet_id = "1721707425"
url = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?gid={sheet_id}&format=csv"
enc_satisfaccion = pd.read_csv(url)
enc_satisfaccion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 14 columns):
 #   Column                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                         --------------  -----  
 0   Marca temporal                                                                                                                                 227 non-null    object 
 1   Por favor ingrese el número de Ticket solo con dígitos.                                                                                        55 non-null     float64
 2   ¿Qué técnicos hicieron parte de este ticket?                                                                                                   37 non-null     object 
 3   ¿Usuario contesta llamada?    

In [14]:
def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False

In [15]:
#Depuro variables y acomodo id ticket
enc_satisfaccion.loc[
    :,'id_ticket']=enc_satisfaccion.loc[
    enc_satisfaccion.iloc[:,7].map(isnumber)].iloc[:,7].combine_first(
        enc_satisfaccion.loc[enc_satisfaccion.iloc[:,10].map(isnumber)].iloc[:,10])

base_satisfaccion=enc_satisfaccion.loc[
    enc_satisfaccion['id_ticket'].notna()].iloc[
    :,[0,-1,4,5,6,9]]

base_satisfaccion.loc[
    :,'id_ticket']=base_satisfaccion[
    'id_ticket'].astype(float).astype(int)

base_satisfaccion.columns=['fecha_enc_sat','id_ticket',
                           'calificacion_personal','resolvio',
                           'promoter','calificacion_servicio']
base_satisfaccion.info()

<class 'pandas.core.frame.DataFrame'>
Index: 170 entries, 56 to 226
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fecha_enc_sat          170 non-null    object 
 1   id_ticket              170 non-null    object 
 2   calificacion_personal  170 non-null    float64
 3   resolvio               170 non-null    object 
 4   promoter               170 non-null    object 
 5   calificacion_servicio  140 non-null    float64
dtypes: float64(2), object(4)
memory usage: 9.3+ KB


In [16]:
for v in list(base_satisfaccion.iloc[:,[3,4]].columns):
    base_satisfaccion.loc[
        :,v
    ]=(base_satisfaccion[v]=='Sí')*1

base_satisfaccion.head()

,fecha_enc_sat,id_ticket,calificacion_personal,resolvio,promoter,calificacion_servicio
56,13/01/2024 10:15:47,19016,5.0,1,1,NaN
57,13/01/2024 10:21:23,19025,5.0,1,1,NaN
58,13/01/2024 10:25:16,19048,5.0,1,1,NaN
59,13/01/2024 10:27:47,19085,5.0,1,1,NaN
60,15/01/2024 14:23:11,19125,5.0,1,1,NaN


## Setup

In [17]:
#Pego con base de usuarios
tickets_append[
    'nombre']=tickets_append[
    'nombre'].str.strip()

base_usuarios[
    'nombre']=base_usuarios[
    'nombre'].str.strip()

working_data=tickets_append.merge(
    base_usuarios, on='nombre', how='left', 
    indicator=True).merge(
    base_material, on='id_ticket', how='left', 
    indicator='merge2').merge(
    base_satisfaccion,on='id_ticket',how='left',
    indicator='merge_sat')

working_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16086 entries, 0 to 16085
Data columns (total 66 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   id_ticket              16086 non-null  object  
 1   departamento           16086 non-null  object  
 2   nombre                 16086 non-null  object  
 3   asunto                 16086 non-null  object  
 4   tecnico                16086 non-null  object  
 5   zona                   16046 non-null  object  
 6   abierto_por            16086 non-null  object  
 7   instal                 16086 non-null  int64   
 8   soporte                16086 non-null  int64   
 9   traslado               16086 non-null  int64   
 10  retiro                 16086 non-null  int64   
 11  mantenimiento          16086 non-null  int64   
 12  facturacion            16086 non-null  int64   
 13  cambio_plan            16086 non-null  int64   
 14  suspendido             16086 non-null 

In [18]:
#Creo variables de ajuste
working_data.loc[:,'encuesta']=working_data['merge2']=='both'*1

#Creo variable de residencial
working_data.loc[:,'residencial']=(working_data['Tipo estrato']!=4)*1

#Calculo tiempo de respuesta, numero de tickets y respuestas a la encuesta
working_data['tiempo_resol']=working_data['tiempo_resolucion']/ pd.Timedelta(days=1)

In [19]:
filepath=os.path.join(outputs,'Base_tickets_satis_material_usuarios')
working_data.to_csv(filepath+'.csv')